#### This notebook demonstrates the use of an odds-equalizing post-processing algorithm for bias mitigiation.


In [1]:
%matplotlib inline
# Load all necessary packages
import sys
import numpy as np
import pandas as pd

sys.path.append("../")
from aif360.datasets import AdultDataset, GermanDataset, CompasDataset
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector

from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions\
                import load_preproc_data_adult, load_preproc_data_compas

from sklearn.preprocessing import scale
from sklearn.linear_model import LogisticRegression

from IPython.display import Markdown, display
import matplotlib.pyplot as plt
from tqdm import tqdm

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

pip install 'aif360[LawSchoolGPA]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[AdversarialDebiasing]'
pip install 'aif360[Reductions]'
pip install 'aif360[Reductions]'
pip install 'aif360[Reductions]'


### Fairness metrics for original dataset

In [20]:
## import dataset
dataset_used = "compas" # "adult", "german", "compas"
protected_attribute_used = 1 # 1, 2

if dataset_used == "adult":
    dataset_orig = AdultDataset()
#     dataset_orig = load_preproc_data_adult()
    if protected_attribute_used == 1:
        privileged_groups = [{'sex': 1}]
        unprivileged_groups = [{'sex': 0}]
    else:
        privileged_groups = [{'race': 1}]
        unprivileged_groups = [{'race': 0}]
    
elif dataset_used == "german":
    dataset_orig = GermanDataset()
    if protected_attribute_used == 1:
        privileged_groups = [{'sex': 1}]
        unprivileged_groups = [{'sex': 0}]
    else:
        privileged_groups = [{'age': 1}]
        unprivileged_groups = [{'age': 0}]
    
elif dataset_used == "compas":
#     dataset_orig = CompasDataset()
    dataset_orig = load_preproc_data_compas()
    if protected_attribute_used == 1:
        privileged_groups = [{'sex': 1}]
        unprivileged_groups = [{'sex': 0}]
    else:
        privileged_groups = [{'race': 1}]
        unprivileged_groups = [{'race': 0}]    

# cost constraint of fnr will optimize generalized false negative rates, that of
# fpr will optimize generalized false positive rates, and weighted will optimize
# a weighted combination of both
cost_constraint = "fnr" # "fnr", "fpr", "weighted"
#random seed for calibrated equal odds prediction
randseed = 12345679 

In [3]:
# Functions

def block(n, k, alpha):
    if(alpha <= 0):
        return 0
    ans = 0
    temp = 0
    for i in range(k + 1, n + 1):
        temp += np.log(i)
    for i in range(1, n - k + 1):
        temp -= np.log(i)
    temp = temp + k * np.log(alpha) + (n - k) * np.log(1 - alpha)
    ans += np.exp(temp)
    if(k < n):
        for j in range(k + 1, n + 1):
            temp = temp + np.log(n - j + 1) -np.log(j) + np.log(alpha) - np.log(1 - alpha)
            ans += np.exp(temp)
    return ans

# def L(n0, n1, k0, k1, alpha, delta):
#     ans = 0
#     N = 100
#     for i in range(N):
#         q = np.random.beta(k1, n1 - k1 + 1)
#         ans += block(n0, k0, q - alpha)
#         if(ans > delta * N):
#             return 1
#     for i in range(N):
#         q = np.random.beta(k0, n0 - k0 + 1)
#         ans += block(n1, k1, q - alpha)
#         if(ans > delta * N):
#             return 1
#     ans = ans / N
#     return ans

# def L_(n0, n1, k0, k1, alpha, delta):
#     ans = 0
#     N = 100
#     for i in range(N):
#         q = 1
#         if(n1 > k1):
#             q = np.random.beta(k1 + 1, n1 - k1)
#         ans += block(n0, k0, q - alpha)
#         if(ans > delta * N):
#             return 1
#     for i in range(N):
#         q = 1
#         if(n0 > k0):
#             q = np.random.beta(k0 + 1, n0 - k0)
#         ans += block(n1, k1, q - alpha)
#         if(ans > delta * N):
#             return 1
#     ans = ans / N
#     return ans

def L(n0, n1, k0, k1, alpha, delta):
    ans = 0
    N = 10000
    for i in range(N):
        qq = np.random.beta(k1, n1 - k1 + 1)
        pp = np.random.beta(k0, n0 - k0 + 1)
        if(abs(qq - pp) > alpha):
            ans += 1
        if(ans > delta * N):
            return 1
    return ans * 1.0 / N

def L_(n0, n1, k0, k1, alpha, delta):
    ans = 0
    N = 10000
    for i in range(N):
        qq = np.random.beta(k1, n1 - k1 + 1)
        pp = np.random.beta(k0, n0 - k0 + 1)
        qq_ = np.random.beta(k1 + 1, n1 - k1)
        pp_ = np.random.beta(k0 + 1, n0 - k0)
        if(qq - pp_ > alpha or pp - qq_ > alpha):
            ans += 1
        if(ans > delta * N):
            return 1
    return ans * 1.0 / N

def M(k_00, k_01, k_10, k_11, n_00, n_01, n_10, n_11):
    return k_10 / (n_10 + 1) * (1 - p1) * p_y0 + k_11 / (n_11 + 1) * p1 * p_y1 + (n_00 + 0.5 - k_00) / (n_00 + 1) * (1 - p1) * (1 - p_y0) + (n_01 + 0.5 - k_01) / (n_01 + 1) * p1 * (1 - p_y1)

def DEOO(th_0, th_1):
    c10 = 0
    c11 = 0
    C10 = 0
    C11 = 0
    for i in range(len(te_10)):
        if(te_10[i] > 0.5):
            c10 += 1
        if(te_10[i] > th_0):
            C10 += 1
    for i in range(len(te_11)):
        if(te_11[i] > 0.5):
            c11 += 1
        if(te_11[i] > th_1):
            C11 += 1
    DEOO = c10 / len(te_10) - c11 / len(te_11)
    DEOO_ = C10 / len(te_10) - C11 / len(te_11)
    ans = np.zeros(2)
    ans[0] = DEOO
    ans[1] = DEOO_
    return ans

def DPE(th_0, th_1):
    c00 = 0
    c01 = 0
    C00 = 0
    C01 = 0
    for i in range(len(te_00)):
        if(te_00[i] > 0.5):
            c00 += 1
        if(te_00[i] > th_0):
            C00 += 1
    for i in range(len(te_01)):
        if(te_01[i] > 0.5):
            c01 += 1
        if(te_01[i] > th_1):
            C01 += 1
    DPE = c00 / len(te_00) - c01 / len(te_01)
    DPE_ = C00 / len(te_00) - C01 / len(te_01)
    ans = np.zeros(2)
    ans[0] = DPE
    ans[1] = DPE_
    return ans

def mis(th_0, th_1):
    c10_ = 0
    c11_ = 0
    C10_ = 0
    C11_ = 0
    for i in range(len(te_10)):
        if(te_10[i] < 0.5):
            c10_ += 1
        if(te_10[i] < th_0):
            C10_ += 1
    for i in range(len(te_11)):
        if(te_11[i] < 0.5):
            c11_ += 1
        if(te_11[i] < th_1):
            C11_ += 1
    for i in range(len(te_00)):
        if(te_00[i] > 0.5):
            c10_ += 1
        if(te_00[i] > th_0):
            C10_ += 1
    for i in range(len(te_01)):
        if(te_01[i] > 0.5):
            c11_ += 1
        if(te_01[i] > th_1):
            C11_ += 1
    mis = (c10_ + c11_) / (len(te_10) + len(te_11) + len(te_00) + len(te_01))
    mis_ = (C10_ + C11_) / (len(te_10) + len(te_11) + len(te_00) + len(te_01))
    ans = np.zeros(2)
    ans[0] = mis
    ans[1] = mis_
    return ans

def E(t):
    ans1 = 0
    ans2 = 0
    for i in range(n_11):
        if(t_11[i] > p1 * p_y1 / (2 * p1 * p_y1 - t)):
            ans1 += 1
    for i in range(n_10):
        if(t_10[i] > (1 - p1) * p_y0 / (2 * (1 - p1) * p_y0 + t)):
            ans2 += 1
    return ans1 / n_11 - ans2 / n_10

In [7]:
L_new = np.load("L_new_62_274.npy")

In [21]:
scale_orig = StandardScaler()
X = scale_orig.fit_transform(dataset_orig.features)
l_00 = np.where((dataset_orig.protected_attributes[:,1] == 0)& (dataset_orig.labels[:,0] == 0))
l_01 = np.where((dataset_orig.protected_attributes[:,1] == 1)& (dataset_orig.labels[:,0] == 0))
l_10 = np.where((dataset_orig.protected_attributes[:,1] == 0)& (dataset_orig.labels[:,0] == 1))
l_11 = np.where((dataset_orig.protected_attributes[:,1] == 1)& (dataset_orig.labels[:,0] == 1))
x_00 = X[l_00[0]]
x_01 = X[l_01[0]]
x_10 = X[l_10[0]]
x_11 = X[l_11[0]]
y_00 = np.zeros(len(x_00), dtype = int)
y_01 = np.zeros(len(x_01), dtype = int)
y_10 = np.ones(len(x_10), dtype = int)
y_11 = np.ones(len(x_11), dtype = int)
X_0 = np.append(x_00, x_10, axis = 0)
Y_0 = np.append(y_00, y_10)
X_1 = np.append(x_01, x_11, axis = 0)
Y_1 = np.append(y_01, y_11)

In [25]:
DEOO_fin = [[], [], [], [], [], []]
DPE_fin = [[], [], [], [], [], []]
mis_fin = [[], [], [], [], [], []]
DEOO_fin_zeng = [[], [], [], [], [], []]
DPE_fin_zeng = [[], [], [], [], [], []]
mis_fin_zeng = [[], [], [], [], [], []]
delta = 0.1
for step in tqdm(range(10)):
    X_tr_00, X_te_00, Y_tr_00, Y_te_00 = train_test_split(x_00, y_00, test_size = 0.2)
    X_tr_01, X_te_01, Y_tr_01, Y_te_01 = train_test_split(x_01, y_01, test_size = 0.2)
    X_tr_10, X_te_10, Y_tr_10, Y_te_10 = train_test_split(x_10, y_10, test_size = 0.2)
    X_tr_11, X_te_11, Y_tr_11, Y_te_11 = train_test_split(x_11, y_11, test_size = 0.2)
    X_tr_00r, X_tr_00c, Y_tr_00r, Y_tr_00c = train_test_split(X_tr_00, Y_tr_00, test_size = 0.7)
    X_tr_01r, X_tr_01c, Y_tr_01r, Y_tr_01c = train_test_split(X_tr_01, Y_tr_01, test_size = 0.7)
    X_tr_10r, X_tr_10c, Y_tr_10r, Y_tr_10c = train_test_split(X_tr_10, Y_tr_10, test_size = 0.7)
    X_tr_11r, X_tr_11c, Y_tr_11r, Y_tr_11c = train_test_split(X_tr_11, Y_tr_11, test_size = 0.7)
    X_tr_0 = np.append(X_tr_00, X_tr_10, axis = 0)
    Y_tr_0 = np.append(Y_tr_00, Y_tr_10)
    X_tr_1 = np.append(X_tr_01, X_tr_11, axis = 0)
    Y_tr_1 = np.append(Y_tr_01, Y_tr_11)
    X_tr_0r = np.append(X_tr_00r, X_tr_10r, axis = 0)
    Y_tr_0r = np.append(Y_tr_00r, Y_tr_10r)
    X_tr_1r = np.append(X_tr_01r, X_tr_11r, axis = 0)
    Y_tr_1r = np.append(Y_tr_01r, Y_tr_11r)
    X_tr_0c = np.append(X_tr_00c, X_tr_10c, axis = 0)
    Y_tr_0c = np.append(Y_tr_00c, Y_tr_10c)
    X_tr_1c = np.append(X_tr_01c, X_tr_11c, axis = 0)
    Y_tr_1c = np.append(Y_tr_01c, Y_tr_11c)
    X_tr_r = np.append(X_tr_0r, X_tr_1r, axis = 0)
    Y_tr_r = np.append(Y_tr_0r, Y_tr_1r)
    X_tr_c = np.append(X_tr_0c, X_tr_1c, axis = 0)
    Y_tr_c = np.append(Y_tr_0c, Y_tr_1c)
    X_te_0 = np.append(X_te_00, X_te_10, axis = 0)
    Y_te_0 = np.append(Y_te_00, Y_te_10)
    X_te_1 = np.append(X_te_01, X_te_11, axis = 0)
    Y_te_1 = np.append(Y_te_01, Y_te_11)
    X_te = np.append(X_te_0, X_te_1, axis = 0)
    Y_te = np.append(Y_te_0, Y_te_1)
    X_tr = np.append(X_tr_0, X_tr_1, axis = 0)
    Y_tr = np.append(Y_tr_0, Y_tr_1)
    log_reg = LogisticRegression(max_iter = 10000)
    log_reg_ = LogisticRegression(max_iter = 10000)
    clm = log_reg.fit(X_tr_r, Y_tr_r)
    clm_ = log_reg_.fit(X_tr, Y_tr)
    n_00 = 0
    n_01 = 0
    n_10 = 0
    n_11 = 0
    t_00 = []
    t_01 = []
    t_10 = []
    t_11 = []
    prob_0 = clm_.predict_proba(X_tr_0)
    prob_1 = clm_.predict_proba(X_tr_1)
    for i in range(len(X_tr_0)):
        if(Y_tr_0[i] == 1):
            n_10 += 1
            t_10.append(prob_0[i, 1])
        else:
            n_00 += 1
            t_00.append(prob_0[i, 1])
    for i in range(len(X_tr_1)):
        if(Y_tr_1[i] == 1):
            n_11 += 1
            t_11.append(prob_1[i, 1])
        else:
            n_01 += 1
            t_01.append(prob_1[i, 1])
    t_00 = np.array(t_00)
    t_01 = np.array(t_01)
    t_10 = np.array(t_10)
    t_11 = np.array(t_11)
    th_0 = np.zeros(6)
    th_1 = np.zeros(6)
    p1 = (n_01 + n_11) / (n_01 + n_11 + n_10 + n_00)
    p_y0 = n_10 / (n_10 + n_00)
    p_y1 = n_11 / (n_11 + n_01)
    for aa in range(6):
        alpha = aa * 0.02 + 0.06
        t = 0
        if(E(0) <= alpha and E(0) + alpha >= 0):
            t = 0
        elif(E(0) > alpha):
            for i in range(500):
                if(E(0.001 * (i + 1)) <= alpha):
                    t = 0.001 * i
                    break
        elif(E(0) + alpha < 0) :
            for i in range(500):
                if(E(-0.001 * (i + 1)) + alpha >= 0):
                    t = -0.001 * (i + 1)
                    break
        th_0[aa] = (1 - p1) * p_y0 / (2 * (1 - p1) * p_y0 + t)
        th_1[aa] = p1 * p_y1 / (2 * p1 * p_y1 - t)
    te_00 = []
    te_01 = []
    te_10 = []
    te_11 = []
    for i in range(len(X_te_0)):
        if(Y_te_0[i] == 1):
            te_10.append(clm_.predict_proba(X_te_0)[i, 1])
        else:
            te_00.append(clm_.predict_proba(X_te_0)[i, 1])
    for i in range(len(X_te_1)):
        if(Y_te_1[i] == 1):
            te_11.append(clm_.predict_proba(X_te_1)[i, 1])
        else:
            te_01.append(clm_.predict_proba(X_te_1)[i, 1])
    te_00 = np.array(te_00)
    te_01 = np.array(te_01)
    te_10 = np.array(te_10)
    te_11 = np.array(te_11)
    for aa in range(6):
        DEOO_fin_zeng[aa].append(DEOO(th_0[aa], th_1[aa])[1])
        DPE_fin_zeng[aa].append(DPE(th_0[aa], th_1[aa])[1])
        mis_fin_zeng[aa].append(mis(th_0[aa], th_1[aa])[1])
    n_00 = 0
    n_01 = 0
    n_10 = 0
    n_11 = 0
    t_00 = []
    t_01 = []
    t_10 = []
    t_11 = []
    prob_0 = clm.predict_proba(X_tr_0c)
    prob_1 = clm.predict_proba(X_tr_1c)
    for i in range(len(X_tr_0c)):
        if(Y_tr_0c[i] == 1):
            n_10 += 1
            t_10.append(prob_0[i, 1])
        else:
            n_00 += 1
            t_00.append(prob_0[i, 1])
    for i in range(len(X_tr_1c)):
        if(Y_tr_1c[i] == 1):
            n_11 += 1
            t_11.append(prob_1[i, 1])
        else:
            n_01 += 1
            t_01.append(prob_1[i, 1])
    t_00 = np.array(t_00)
    t_01 = np.array(t_01)
    t_10 = np.array(t_10)
    t_11 = np.array(t_11)
    t_00 = np.sort(t_00)
    t_01 = np.sort(t_01)
    t_10 = np.sort(t_10)
    t_11 = np.sort(t_11)
    p1 = (n_01 + n_11) / (n_01 + n_11 + n_10 + n_00)
    p_y0 = n_10 / (n_10 + n_00)
    p_y1 = n_11 / (n_11 + n_01)
    cnt = np.zeros(6)
    min = np.ones(6) * 10000
    th_0 = -1 * np.ones(6)
    th_1 = -1 * np.ones(6)
    for k_10 in range(1, n_10 + 1):
        mmin = 10000
        for j in range(1, n_11 + 1):
            val = 1 / (2 - (1 - p1) * p_y0 * (1 / t_10[k_10 - 1] - 2) / (p1 * p_y1))
            tp = abs(t_11[j - 1] - val)
            if(tp < mmin):
                mmin = tp
                k_11 = j
        k_00 = n_00
        if(t_00[0] > t_10[k_10 - 1]):
            k_00 = 0
        for i in range(n_00 - 1):
            if(t_00[i] <= t_10[k_10 - 1] and t_00[i + 1] > t_10[k_10 - 1]):
                k_00 = i + 1
                break
        k_01 = n_01
        if(t_01[0] > t_11[k_11 - 1]):
            k_01 = 0
        for i in range(n_01 - 1):
            if(t_01[i] <= t_11[k_11 - 1] and t_01[i + 1] > t_11[k_11 - 1]):
                k_01 = i + 1
                break
        if(k_01 != 0 and k_01 != n_01 and k_00 != 0 and k_00 != n_00):
            for aa in range(6):
                alpha = aa * 0.02 + 0.06
                if(L(n_10, n_11, k_10, k_11, alpha, delta) < 1):
                    if(L_(n_00, n_01, k_00, k_01, alpha, delta) < 1):
                        cnt[aa] += 1
                        if(M(k_00, k_01, k_10, k_11, n_00, n_01, n_10, n_11) < min[aa]):
                            min[aa] = M(k_00, k_01, k_10, k_11, n_00, n_01, n_10, n_11)
                            th_0[aa] = t_10[k_10 - 1]
                            th_1[aa] = t_11[k_11 - 1]
    te_00 = []
    te_01 = []
    te_10 = []
    te_11 = []
    for i in range(len(X_te_0)):
        if(Y_te_0[i] == 1):
            te_10.append(clm.predict_proba(X_te_0)[i, 1])
        else:
            te_00.append(clm.predict_proba(X_te_0)[i, 1])
    for i in range(len(X_te_1)):
        if(Y_te_1[i] == 1):
            te_11.append(clm.predict_proba(X_te_1)[i, 1])
        else:
            te_01.append(clm.predict_proba(X_te_1)[i, 1])
    te_00 = np.array(te_00)
    te_01 = np.array(te_01)
    te_10 = np.array(te_10)
    te_11 = np.array(te_11)
    for aa in range(6):
        if(th_0[aa] > -1):
            DEOO_fin[aa].append(DEOO(th_0[aa], th_1[aa])[1])
            DPE_fin[aa].append(DPE(th_0[aa], th_1[aa])[1])
            mis_fin[aa].append(mis(th_0[aa], th_1[aa])[1])
        # print(DEOO(th_0[aa], th_1[aa])[1])
        # print(DPE(th_0[aa], th_1[aa])[1])
        # print(mis(th_0[aa], th_1[aa])[1])
        # print(min[aa])
        # print(cnt[aa])
DEOO_fin = np.array(DEOO_fin)
DPE_fin = np.array(DPE_fin)
mis_fin = np.array(mis_fin)
DEOO_fin_zeng = np.array(DEOO_fin_zeng)
DPE_fin_zeng = np.array(DPE_fin_zeng)
mis_fin_zeng = np.array(mis_fin_zeng)
# np.save("DEOO_eq_ours_111.npy", DEOO_fin)
# np.save("DPE_eq_ours_111.npy", DPE_fin)
# np.save("mis_eq_ours_111.npy", mis_fin)
# np.save("DEOO_odds_zeng_111.npy", DEOO_fin_zeng)
# np.save("DPE_odds_zeng_111.npy", DPE_fin_zeng)
# np.save("mis_odds_zeng_111.npy", mis_fin_zeng)

100%|██████████| 10/10 [09:43<00:00, 58.37s/it]


In [26]:
for i in range(6):
    DEOO_fin[i] = np.array(DEOO_fin[i])
    mis_fin[i] = np.array(mis_fin[i])
    DEOO_fin_zeng[i] = np.array(DEOO_fin_zeng[i])
    mis_fin_zeng[i] = np.array(mis_fin_zeng[i])
    DPE_fin[i] = np.array(DPE_fin[i])
    DPE_fin_zeng[i] = np.array(DPE_fin_zeng[i])


In [23]:
DEOO_fin_ = [[], [], [], [], [], []]
DPE_fin_ = [[], [], [], [], [], []]
mis_fin_ = [[], [], [], [], [], []]
DEOO_fin_zeng_ = [[], [], [], [], [], []]
DPE_fin_zeng_ = [[], [], [], [], [], []]
mis_fin_zeng_ = [[], [], [], [], [], []]
for i in range(6):
    DEOO_fin_[i] = DEOO_fin[i][:497]
    DPE_fin_[i] = DPE_fin[i][:497]
    mis_fin_[i] = mis_fin[i][:497]
    DEOO_fin_zeng_[i] = DEOO_fin_zeng[i][:497]
    DPE_fin_zeng_[i] = DPE_fin_zeng[i][:497]
    mis_fin_zeng_[i] = mis_fin_zeng[i][:497]
DEOO_fin_ = np.array(DEOO_fin_)
DPE_fin_ = np.array(DPE_fin_)
mis_fin_ = np.array(mis_fin_)
DEOO_fin_zeng_ = np.array(DEOO_fin_zeng_)
DPE_fin_zeng_ = np.array(DPE_fin_zeng_)
mis_fin_zeng_ = np.array(mis_fin_zeng_)

In [24]:
np.save("DEOO_eq_ours_0926_00001.npy", DEOO_fin_)
np.save("DPE_eq_ours_0926_00001.npy", DPE_fin_)
np.save("mis_eq_ours_0926_00001.npy", mis_fin_)
np.save("DEOO_odds_zeng_0926_00001.npy", DEOO_fin_zeng_)
np.save("DPE_odds_zeng_0926_00001.npy", DPE_fin_zeng_)
np.save("mis_odds_zeng_0926_00001.npy", mis_fin_zeng_)

In [27]:
for i in range(6):
    print(np.mean(abs(DEOO_fin[i])))
    print(np.percentile(abs(DEOO_fin[i]), 95))
    print(np.mean(abs(DPE_fin[i])))
    print(np.percentile(abs(DPE_fin[i]),95))
    print(1-np.mean(abs(mis_fin[i])))
    print(np.mean(abs(DEOO_fin_zeng[i])))
    print(np.percentile(abs(DEOO_fin_zeng[i]), 95))
    print(np.mean(abs(DPE_fin_zeng[i])))
    print(np.percentile(abs(DPE_fin_zeng[i]), 95))
    print(1-np.mean(abs(mis_fin_zeng[i])))
    print("haha")

0.01672399672399672
0.06751296751296737
0.02388052034775464
0.05292920342617923
0.5139886578449905
0.059322959322959314
0.10277368277368272
0.0414005213750947
0.07904675681575934
0.6529300567107751
haha
0.055702975702975686
0.13728910728910723
0.05273978759743679
0.08484288117527705
0.6014177693761815
0.05689871689871688
0.10277368277368272
0.039844101141631665
0.07048644553171267
0.6529300567107751
haha
0.09073437073437071
0.15678678678678673
0.048302962591979054
0.09226862374953451
0.6537807183364839
0.05689871689871688
0.10277368277368272
0.039844101141631665
0.07048644553171267
0.6529300567107751
haha
0.09835653835653835
0.15678678678678673
0.05670275198726099
0.13310603433884238
0.6704158790170133
0.05689871689871688
0.10277368277368272
0.039844101141631665
0.07048644553171267
0.6529300567107751
haha
0.11782145782145784
0.1598170898170898
0.059684606592955027
0.11404309691669555
0.6756143667296786
0.08235326235326233
0.1802020202020201
0.05555213108859523
0.10097019429569415
0.653

In [25]:
aa = np.load("DEOO_eq_ours_111.npy")
bb = np.load("DPE_eq_ours_111.npy")
cc = np.load("mis_eq_ours_111.npy")
dd = np.load("DEOO_odds_zeng_111.npy")
ee = np.load("DPE_odds_zeng_111.npy")
ff = np.load("mis_odds_zeng_111.npy")
for i in range(6):
    print(np.percentile(abs(aa[i]),90))
    print(np.mean(abs(bb[i])))
    print(1-np.mean(cc[i]))
    print(np.mean(dd[i]))
    print(np.mean(ee[i]))
    print(1-np.mean(ff[i]))
    print("haha")

0.07676816178962835
0.06547197857986684
0.8457552509396418
0.06599246161764522
nan
0.8474757904045987
haha
0.09502786879072048
0.06984632304898324
0.846024320141499
0.08512463124066334
nan
0.8475467609993368
haha
0.11212449315206068
0.0731404670644977
0.8461908025646695
0.10129509534390338
nan
0.8475666592969269
haha
0.12546385216987413
0.0757734783621335
0.8462657528189255
0.10378923913179929
nan
0.8475677647579041
haha
0.14211953451587397
0.07810637415690036
0.8462891885916427
0.10378923913179929
nan
0.8475677647579041
haha
0.1551745565472828
0.07972731268078172
0.8462787972584568
0.10378923913179929
nan
0.8475677647579041
haha


In [4]:
for i in range(6):
    print(np.percentile(abs(aa[i]), 90))
    print(np.percentile(abs(bb[i]), 90))
    print(1-np.mean(cc[i]))
    print(np.percentile(dd[i], 90))
    print(np.mean(ee[i]))
    print(1-np.mean(ff[i]))
    print("haha")

0.07676816178962835
0.07401110494819556
0.8457552509396418
0.10741473028785195
nan
0.8474757904045987
haha
0.09502786879072048
0.07790269971045673
0.846024320141499
0.1278198320340451
nan
0.8475467609993368
haha
0.11212449315206068
0.08167465997220472
0.8461908025646695
0.1399609585859727
nan
0.8475666592969269
haha
0.12546385216987413
0.08498799084605943
0.8462657528189255
0.1399609585859727
nan
0.8475677647579041
haha
0.14211953451587397
0.08867049945575808
0.8462891885916427
0.1399609585859727
nan
0.8475677647579041
haha
0.1551745565472828
0.09050395667710057
0.8462787972584568
0.1399609585859727
nan
0.8475677647579041
haha


c:\Users\lphLe\anaconda3\envs\aif360\lib\site-packages\numpy\core\fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
c:\Users\lphLe\anaconda3\envs\aif360\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [65]:
DEOO_fin_ = []
DPE_fin_ = []
mis_fin_ = []
for i in range(6):
    DEOO_fin_.append(DEOO_fin[i][:495])
    DPE_fin_.append(DPE_fin[i][:495])
    mis_fin_.append(mis_fin[i][:495])

In [66]:
DEOO_fin_ = np.array(DEOO_fin_)
DPE_fin_ = np.array(DPE_fin_)
mis_fin_ = np.array(mis_fin_)

In [67]:
np.save("DEOO_odds_ours_11.npy", DEOO_fin_)
np.save("DPE_odds_ours_11.npy", DPE_fin_)
np.save("mis_odds_ours_11.npy", mis_fin_)

In [72]:
for i in range(6):
    print(np.mean(abs(DEOO_fin[i])))
    print(np.mean(abs(DPE_fin[i])))
    print(np.mean(abs(mis_fin[i])))
    print(np.mean(abs(DEOO_fin_zeng[i])))
    print(np.mean(abs(DPE_fin_zeng[i])))
    print(np.mean(abs(mis_fin_zeng[i])))
    print("haha")

0.038791448549442005
0.06547197857986684
0.15424474906035818
0.06599246161764522
nan
0.15252420959540128
haha
0.05239253568335029
0.06984632304898324
0.153975679858501
0.08512463124066334
nan
0.15245323900066327
haha
0.06648873950212783
0.0731404670644977
0.15380919743533053
0.10129509534390338
nan
0.15243334070307316
haha
0.07873183193361705
0.0757734783621335
0.15373424718107453
0.10378923913179929
nan
0.15243223524209595
haha
0.09035590454311503
0.07810637415690036
0.1537108114083573
0.10378923913179929
nan
0.15243223524209595
haha
0.0998678115467179
0.07972731268078172
0.15372120274154324
0.10378923913179929
nan
0.15243223524209595
haha


c:\Users\lphLe\anaconda3\envs\aif360\lib\site-packages\numpy\core\fromnumeric.py:3441: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
c:\Users\lphLe\anaconda3\envs\aif360\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


#### Divide dataset into train, validation, and test partitions (70-30)

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve

# Odds equalizing post-processing algorithm
from aif360.algorithms.postprocessing.calibrated_eq_odds_postprocessing import CalibratedEqOddsPostprocessing
from aif360.algorithms.postprocessing.eq_odds_postprocessing import EqOddsPostprocessing
from tqdm import tqdm
import gc

### Train classifier (logistic regression on original training data)

In [4]:
mis = 0
deoo = 0
dpe = 0
for step in tqdm(range(300)):
    dataset_orig_train, dataset_orig_vt = dataset_orig.split([0.6], shuffle=True, seed = np.random.randint(0, 10000))
    dataset_orig_valid, dataset_orig_test = dataset_orig_vt.split([0.5], shuffle=True, seed = np.random.randint(0, 10000))
    # Placeholder for predicted and transformed datasets
    dataset_orig_train_pred = dataset_orig_train.copy(deepcopy=True)
    dataset_orig_valid_pred = dataset_orig_valid.copy(deepcopy=True)
    dataset_orig_test_pred = dataset_orig_test.copy(deepcopy=True)

    dataset_new_valid_pred = dataset_orig_valid.copy(deepcopy=True)
    dataset_new_test_pred = dataset_orig_test.copy(deepcopy=True)

    # Logistic regression classifier and predictions for training data
    scale_orig = StandardScaler()
    X_train = scale_orig.fit_transform(dataset_orig_train.features)
    y_train = dataset_orig_train.labels.ravel()
    lmod = LogisticRegression()
    lmod.fit(X_train, y_train)

    fav_idx = np.where(lmod.classes_ == dataset_orig_train.favorable_label)[0][0]
    y_train_pred_prob = lmod.predict_proba(X_train)[:,fav_idx]

    # Prediction probs for validation and testing data
    X_valid = scale_orig.transform(dataset_orig_valid.features)
    y_valid_pred_prob = lmod.predict_proba(X_valid)[:,fav_idx]

    X_test = scale_orig.transform(dataset_orig_test.features)
    y_test_pred_prob = lmod.predict_proba(X_test)[:,fav_idx]
    y_test = dataset_orig_test.labels.ravel()

    class_thresh = 0.5
    dataset_orig_train_pred.scores = y_train_pred_prob.reshape(-1,1)
    dataset_orig_valid_pred.scores = y_valid_pred_prob.reshape(-1,1)
    dataset_orig_test_pred.scores = y_test_pred_prob.reshape(-1,1)

    y_train_pred = np.zeros_like(dataset_orig_train_pred.labels)
    y_train_pred[y_train_pred_prob >= class_thresh] = dataset_orig_train_pred.favorable_label
    y_train_pred[~(y_train_pred_prob >= class_thresh)] = dataset_orig_train_pred.unfavorable_label
    dataset_orig_train_pred.labels = y_train_pred

    y_valid_pred = np.zeros_like(dataset_orig_valid_pred.labels)
    y_valid_pred[y_valid_pred_prob >= class_thresh] = dataset_orig_valid_pred.favorable_label
    y_valid_pred[~(y_valid_pred_prob >= class_thresh)] = dataset_orig_valid_pred.unfavorable_label
    dataset_orig_valid_pred.labels = y_valid_pred
        
    y_test_pred = np.zeros_like(dataset_orig_test_pred.labels)
    y_test_pred[y_test_pred_prob >= class_thresh] = dataset_orig_test_pred.favorable_label
    y_test_pred[~(y_test_pred_prob >= class_thresh)] = dataset_orig_test_pred.unfavorable_label
    dataset_orig_test_pred.labels = y_test_pred

    # Learn parameters to equalize odds and apply to create a new dataset
    cpp = CalibratedEqOddsPostprocessing(privileged_groups = privileged_groups,
                                         unprivileged_groups = unprivileged_groups,
                                         cost_constraint=cost_constraint,
                                         seed=randseed)
    cpp = cpp.fit(dataset_orig_valid, dataset_orig_valid_pred)

    # cpp = EqOddsPostprocessing(privileged_groups = privileged_groups,
    #                                     unprivileged_groups = unprivileged_groups,
    #                                     seed=randseed)
    # cpp = cpp.fit(dataset_orig_valid, dataset_orig_valid_pred)

    dataset_transf_valid_pred = cpp.predict(dataset_orig_valid_pred)
    dataset_transf_test_pred = cpp.predict(dataset_orig_test_pred)
    eq = 0
    for i in range(len(y_test)):
        if(dataset_transf_test_pred.labels.ravel()[i] == y_test[i]):
            eq += 1
    mis += eq / len(y_test)
    n_10 = 0
    n_11 = 0
    c_10 = 0
    c_11 = 0
    for i in range(len(y_test)):
        if(y_test[i] == 1 and dataset_transf_test_pred.protected_attributes[:,1][i] == 0):
            n_10 += 1
            if(dataset_transf_test_pred.labels.ravel()[i] == 1):
                c_10 += 1
        elif(y_test[i] == 1 and dataset_transf_test_pred.protected_attributes[:,1][i] == 1):
            n_11 += 1
            if(dataset_transf_test_pred.labels.ravel()[i] == 1):
                c_11 += 1
    deoo += abs(c_10 / n_10 - c_11 / n_11)
    n_00 = 0
    n_01 = 0
    c_00 = 0
    c_01 = 0
    for i in range(len(y_test)):
        if(y_test[i] == 0 and dataset_transf_test_pred.protected_attributes[:,1][i] == 0):
            n_00 += 1
            if(dataset_transf_test_pred.labels.ravel()[i] == 1):
                c_00 += 1
        elif(y_test[i] == 0 and dataset_transf_test_pred.protected_attributes[:,1][i] == 1):
            n_01 += 1
            if(dataset_transf_test_pred.labels.ravel()[i] == 1):
                c_01 += 1
    dpe += abs(c_00 / n_00 - c_01 / n_01)
    print(mis)
    print(deoo)
    print(dpe)
print(mis/ 300)
print(deoo / 300)
print(dpe / 300)

  0%|          | 1/300 [00:00<03:05,  1.61it/s]

0.8247650635710337
0.09834368530020704
0.038905104928188244


  1%|          | 2/300 [00:01<03:02,  1.63it/s]

1.6501934770591489
0.13573945721968822
0.08626603906158954


  1%|          | 3/300 [00:01<03:05,  1.60it/s]

...

245.46323935876146
23.874180854016704
13.801416983304899


100%|█████████▉| 299/300 [03:09<00:00,  1.54it/s]

246.2923161967935
23.929926952038375
13.84439534887749


100%|██████████| 300/300 [03:10<00:00,  1.58it/s]

247.10757324488637
24.047606624813525
13.891757335810334
0.8236919108162879
0.08015868874937841
0.046305857786034445


In [ ]:
print(y_test)

### Perform odds equalizing post processing on scores

### Transform validation and test data using the post processing algorithm

In [41]:
n_10 = 0
n_11 = 0
c_10 = 0
c_11 = 0
for i in range(len(y_test)):
    if(y_test[i] == 1 and dataset_orig_test_pred.protected_attributes[:,1][i] == 0):
        n_10 += 1
        if(dataset_orig_test_pred.labels.ravel()[i] == 1):
            c_10 += 1
    elif(y_test[i] == 1 and dataset_orig_test_pred.protected_attributes[:,1][i] == 1):
        n_11 += 1
        if(dataset_orig_test_pred.labels.ravel()[i] == 1):
            c_11 += 1
DEOO = c_10 / n_10 - c_11 / n_11
print(DEOO)

-0.052362089201877904


In [42]:
n_10 = 0
n_11 = 0
c_10 = 0
c_11 = 0
for i in range(len(y_test)):
    if(y_test[i] == 1 and dataset_transf_test_pred.protected_attributes[:,1][i] == 0):
        n_10 += 1
        if(dataset_transf_test_pred.labels.ravel()[i] == 1):
            c_10 += 1
    elif(y_test[i] == 1 and dataset_transf_test_pred.protected_attributes[:,1][i] == 1):
        n_11 += 1
        if(dataset_transf_test_pred.labels.ravel()[i] == 1):
            c_11 += 1
DEOO = c_10 / n_10 - c_11 / n_11
print(DEOO)

0.01676203051643188


In [43]:
n_00 = 0
n_01 = 0
c_00 = 0
c_01 = 0
for i in range(len(y_test)):
    if(y_test[i] == 0 and dataset_orig_test_pred.protected_attributes[:,1][i] == 0):
        n_00 += 1
        if(dataset_orig_test_pred.labels.ravel()[i] == 1):
            c_00 += 1
    elif(y_test[i] == 0 and dataset_orig_test_pred.protected_attributes[:,1][i] == 1):
        n_01 += 1
        if(dataset_orig_test_pred.labels.ravel()[i] == 1):
            c_01 += 1
DPE = c_00 / n_00 - c_01 / n_01
print(DPE)

-0.08315624777398335


In [44]:
n_00 = 0
n_01 = 0
c_00 = 0
c_01 = 0
for i in range(len(y_test)):
    if(y_test[i] == 0 and dataset_transf_test_pred.protected_attributes[:,1][i] == 0):
        n_00 += 1
        if(dataset_transf_test_pred.labels.ravel()[i] == 1):
            c_00 += 1
    elif(y_test[i] == 0 and dataset_transf_test_pred.protected_attributes[:,1][i] == 1):
        n_01 += 1
        if(dataset_transf_test_pred.labels.ravel()[i] == 1):
            c_01 += 1
DPE = c_00 / n_00 - c_01 / n_01
print(DPE)

0.0009294502432419666


In [45]:
DEO_CEP = np.array([0.09917986743062573, 0.043843592084027834])
acc_CEP = 0.814262023217247
DEO_EP = np.array([0.016576638027081934, 0.0009415807062945639])
acc_EP = 0.8185764510779303
413.36561636263207
53.54782099587427
-23.46021170903661